# cube - Erste Schritte mit jupyter und openCV3

Irgendwie wurde es mal Zeit, daß ich mich mit der Verarbeitung von Bildern im Computer jenseits von Gimp und Photoshop beschäftige. Besonders klar wurde mir das, als ich für eine Installation die Kamera und das Infrarotfilter recherchiert und schließlich eine recht aufwändige Hardware angefertigt habe: http://www.bautzner-strasse-dresden.de/neu/ausstellungen.html Doch was ist mit der Software? Diese ist das Werk von http://frauzufall.de Davon verstand ich überhaupt nichts. Und dabei sollte es aber nicht ganz bleiben. Ich wollte mehr darüber wissen, wie Computer, also Maschinen sehen können. Und außerdem war da in meinem Kopf noch so ne alte Idee. Man müsste mal....
Ehrlichgesagt hab ich nicht daran geglaubt, dass ich das jemals umsetzten würde. Aber dann war es erstaunlich leicht. 

Meine ersten Anlaufpunkte waren Python3(der Kleber), numpy(der virtuelle Quader) und openCV3(Die riesige Bildverarbeitungsmaschinerie für C++, Java, Python, die ich dafür nur angekratzt habe). Und jupyter. Das ist eine clevere IDE um zu bauen, zu testen und zu teilen. Leider ist openCV3 noch nicht Teil der direkt verfügbaren Pakete in conda. 
So hat die Installation bei mir funktioniert:

Ich hab pip aktualisiert, die Eingabeaufforderung auch schön mit rechter Maus gestartet (mit Admin-rechten!).
bei Michael Hirsch fand ich eine gute Anleitung 

#### https://www.scivision.co/install-opencv-python-windows/

Das hat auf Anhieb gut funktioniert. Beide Kameras meines Tablets konnten sofort genutzt werden, nur schien der Videoencoder ein Problem mit schwarz-weiss Frames zu haben. Ich hab endlich cv2.cvtColor gegoogled und bin sofort auf "OpenCV: Changing Colorspaces gestoßen" 
Eine erster Anlaufpunkt mit einer sehr guten Dokumentation ist hier: http://docs.opencv.org/ 
Ich hab mich sehr eng an den Beispielen von 3.2.0 orientiert. Hier ist alles knapp und übersichtlich erklärt.
Fehlt noch ne gute Erklärung zu numpy Arrays. Die fand ich bei Wes McKinney. Und bei Leo Chin und Tanmay Dutta.(PACKT) Ich staune. Doch nun zum inhaltlichen. Was soll das?

# cube -  Ein sehr interessanter virtueller Körper

####    Wie würde eine Welt aussehen, in der Zeit keine Rolle spielt, weil jede Zeit direkt zugänglich ist?
####    Wie sieht Zeit aus, wenn man sie von oben betrachtet?
####    Was bedeutet überhaupt Raumzeit?
####    Was ist, wenn man die Scalen für Raum und Zeit miteinander vertauscht?
#  Hab ich das schonmal gesehen?

cube ist eine Versuchsanordnung, eine flache Welt, deren Zeiten alle zugänglich sind. Doch cube ist nicht real. Es existiert nur als Bereich im Arbeitsspeicher des Computers. cube ist ein numpy-3D-Array mit den Abmessungen (480x640x640)x8Bit. Dieses Array enthält 640 chronologische, fortlaufend aktualisierte schwarz-weiss Frames der bevorzugten bordeigenen Kamera. So entsteht ein Quader. Die 3Dimensionen entsprechen Höhe, Breite und Zeit (y,x,t)
Das Tolle ist, dass man sich diesen Quader von verschiedenen Seiten anschauen kann.

# Denn ein Pixel interessiert nicht, von welcher Seite man es betrachtet.

Der unten abgebildete Code erzeugt 2 Videofenster. Die Fenster stellen 2Projektionen des Quaders dar:

1.) Wir sehen alle x und y und was als Film abläuft ist die Zeit (ein ganz normaler Film also, so wie gewohnt).

2.) Wir sehen alle Zeiten und y und was als Film abläuft ist x (das Gehirn tut weh, aber die Augen entspannen)

In [ ]:
#GREY VERSION WITHOUT ROTATE
import numpy as np
import cv2
#using frontCamera: source = 1
#using file:        source = 'path.avi'
source = 0    # backCamera
camerastream = cv2.VideoCapture(source)
#Define the codec and create VideoWriter object note: isColor is False for Gray
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out2stream = cv2.VideoWriter('out2.avi',fourcc, 20.0, (640,480), isColor = False)
#build cube with (y,x,t)*8Bit
cube = np.zeros((480,640,640),dtype = np.uint8)

play_x_foreward = True
rec_t = 0
play_x = 0
while True:
    #take each frame from camerastream
    _, orginalframe = camerastream.read() # "_," häh ??? aber notwendig
    #convert from BGR to GRAY
    grayframe = cv2.cvtColor(orginalframe, cv2.COLOR_BGR2GRAY)
    #load videoframe to 2D+TimeCube
    cube[:,:,rec_t] = grayframe
    #build outputframe use magic rolling view(y,t,x) is faster then cube = np.roll(cube,1,axis=2)
    outputframe = np.concatenate([cube[:,play_x,rec_t:], cube[:,play_x,:rec_t]], axis=1)
    
    #calc next rec_t and play_x
    rec_t += 1
    if rec_t >= cube.shape[2]:
        play_x_foreward = False
        rec_t = 0
        
    if play_x_foreward == True:
        play_x += 1
    else: 
        play_x -= 1
        
    if play_x <= 0:
        play_x_foreward = True
        play_x = 0
    
    #show images in 2 windows (gray, output)
    cv2.imshow('gray', grayframe)
    cv2.imshow('output', outputframe)
    
    #write stream to .avi -file
    out2stream.write(outputframe)
    
    #pause 1ms (wait for space-key (0 = endless, 1 = 1ms, 0xFF need only for 64Bit CPU)) 
    if cv2.waitKey(1) & 0xFF == ord(' '):
        #for debug only
        print("orginalframe.shape  .dtype:",  orginalframe.shape,"   ",orginalframe.dtype)
        print("        cube.shape  .dtype:",                    cube.shape," ",cube.dtype)
        print(" outputframe.shape  .dtype:", outputframe.shape,"      ",outputframe.dtype)
        break

# release everything if job is finished
camerastream.release()
out2stream.release()
cv2.destroyAllWindows()

In [12]:
#GREY VERSION WITH ROTATE
import numpy as np
import cv2
#using frontCamera: source = 1
#using file:        source = 'path.avi'
source = 0    # backCamera
camerastream = cv2.VideoCapture(source)
#Define the codec and create VideoWriter object note: isColor is False for Gray
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out2stream = cv2.VideoWriter('out2.avi',fourcc, 20.0, (640,480), isColor = False)
#build cube with (y,x,t)*8Bit
cube = np.zeros((480,640,640),dtype = np.uint8)

play_x_foreward = True
rec_t = 0
play_x = 0
while True:
    #take each frame from camerastream
    _, orginalframe = camerastream.read() # "_," häh ??? aber notwendig
    #convert from BGR to GRAY
    grayframe = cv2.cvtColor(orginalframe, cv2.COLOR_BGR2GRAY)
    #load videoframe to 2D+TimeCube
    cube[:,:,rec_t] = grayframe
    #build outputframe use magic rolling view(y,t,x) is faster then cube = np.roll(cube,1,axis=2)
    outputframe = np.concatenate([cube[:,play_x,rec_t:], cube[:,play_x,:rec_t]], axis=1)
    
    #calc next rec_t and play_x
    rec_t += 1
    if rec_t >= cube.shape[2]:
        play_x_foreward = False
        rec_t = 0
        
    if play_x_foreward == True:
        play_x += 1
    else: 
        play_x -= 1
        
    if play_x <= 0:
        play_x_foreward = True
        play_x = 0
    
    #rotate
    grayframe90cw = cv2.rotate(grayframe, cv2.ROTATE_90_CLOCKWISE)
    outputframe90cw = cv2.rotate(outputframe, cv2.ROTATE_90_CLOCKWISE)
    #show images in 2 windows (gray, output)
    cv2.imshow('gray', grayframe90cw)
    cv2.imshow('output', outputframe90cw)
    
    #write stream to .avi -file
    out2stream.write(outputframe)
    
    #pause 1ms (wait for space-key (0 = endless, 1 = 1ms, 0xFF need only for 64Bit CPU)) 
    if cv2.waitKey(1) & 0xFF == ord(' '):
        #for debug only
        print("orginalframe.shape  .dtype:",  orginalframe.shape,"   ",orginalframe.dtype)
        print("        cube.shape  .dtype:",                    cube.shape," ",cube.dtype)
        print(" outputframe.shape  .dtype:", outputframe.shape,"      ",outputframe.dtype)
        break

# release everything if job is finished
camerastream.release()
out2stream.release()
cv2.destroyAllWindows()

orginalframe.shape  .dtype: (480, 640, 3)     uint8
        cube.shape  .dtype: (480, 640, 640)   uint8
 outputframe.shape  .dtype: (480, 640)        uint8


In [1]:
# COLOR VERSION WITH ROTATE
import numpy as np
import cv2
#using frontCamera: source = 1
#using file:        source = 'path.avi'
source = 0    # backCamera
camerastream = cv2.VideoCapture(source)
#Define the codec and create VideoWriter object note: isColor is True
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out2stream = cv2.VideoWriter('out2.avi',fourcc, 20.0, (640,480), isColor = True)
#build cube with (y,x,t)*3*8Bit
cube = np.zeros((480,640,640,3),dtype = np.uint8)

play_x_foreward = True
rec_t = 0
play_x = 0
while True:
    #take each frame from camerastream
    _, frame = camerastream.read() # "_," häh ??? aber notwendig
    #load videoframe to 2D+TimeCube
    cube[:,:,rec_t,:] = frame
    #build outputframe use magic rolling view(y,t,x) is faster then cube = np.roll(cube,1,axis=2)
    outputframe = np.concatenate([cube[:,play_x,rec_t:,:], cube[:,play_x,:rec_t,:]], axis=1)
    
    #calc next rec_t and play_x
    rec_t += 1
    if rec_t >= cube.shape[2]:
        play_x_foreward = False
        rec_t = 0
        
    if play_x_foreward == True:
        play_x += 1
    else: 
        play_x -= 1
        
    if play_x <= 0:
        play_x_foreward = True
        play_x = 0
    
    #rotate
    frame90cw = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    outputframe90cw = cv2.rotate(outputframe, cv2.ROTATE_90_CLOCKWISE)
    #show images in 2 windows (gray, output)
    cv2.imshow('input', frame90cw)
    cv2.imshow('output', outputframe90cw)
    
    #write stream to .avi -file
    out2stream.write(outputframe)
    
    #pause 1ms (wait for space-key (0 = endless, 1 = 1ms, 0xFF need only for 64Bit CPU)) 
    if cv2.waitKey(1) & 0xFF == ord(' '):
        #for debug only
        print("       frame.shape  .dtype:",frame.shape,      "       ",frame.dtype)
        print("        cube.shape  .dtype:",cube.shape,       "  ",cube.dtype)
        print(" outputframe.shape  .dtype:",outputframe.shape,"       ",outputframe.dtype)
        break

# release everything if job is finished
camerastream.release()
out2stream.release()
cv2.destroyAllWindows()

       frame.shape  .dtype: (480, 640, 3)         uint8
        cube.shape  .dtype: (480, 640, 640, 3)    uint8
 outputframe.shape  .dtype: (480, 640, 3)         uint8


# Wie würde das mechanisch aussehen, wenn man es in einer Werkstatt anfertigt?

Leider hab ich da noch keine Idee. Soetwas scheint nur in diesem virtuellen Quader möglich zu sein. Jedoch sinngemäß funktioniert das wie bei einem Daumenkino. Wir stellen uns mal vor, dass wir fleissig waren und auf so einen Bürowürfel voller Klebezettel einen ganzen Trickfilm gepinselt haben. Zum Angucken können wir die Blätter am Daumen vorbeirutschen lassen...soweit kennen wir das. Aber jetzt drehen wir den Würfel mal um 90Grad und schauen ihn uns von der Seite an. Wir sehen vielleicht etwas Bleistift, wenn wir in der Eile beim Zeichnen über den Rand des Papiers hinaus gemalt haben. Wenn wir aber Schicht für Schicht den Würfel abhobeln würden, dann sähen wir das: (Den dabei entstehenden Film zeichnet der Code auch als out2.avi auf.) Genaugenommen pendelt der Film beim Angucken auf der x-Achse hin und her.

Es gibt aber auch noch andere Anordnungen, in denen ein Zeitausschnitt als Ganzes abgebildet wird. Nur eben nicht 3-dimensional wie in cube:

#### #Schlitzverschluss
#### #Scannerkamera
#### #Slitcam
#### #Oszilloscope